In [1]:
import matplotlib 
import os
import glob
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [2]:
os.chdir('Pokemon')
pokeArray = np.empty((0, 40, 40, 3), dtype='float32')
for pokemon in glob.glob('*.png'):
    img=mpimg.imread(pokemon)
    img[img[:,:,3]==0] = np.ones((1,4))
    img = img[:,:,0:3]
    pokeArray=np.append(pokeArray,[img],axis=0)
os.chdir('./..')

In [3]:
# Parameters
learning_rate = 0.005#learning rate changed to 0.05 from 0.01
training_epochs = 1000
batch_size = 24
display_step = 100
examples_to_show = 8

# Network Parameters
n_input = [40, 40, 3] # Pokemon data input (img shape: 40*40*3)
n_channel1 = 32#modified from 16 to 32 to see changes
n_channel2 = 64#modified from 32 to 64 to see changes
X = tf.placeholder(tf.float32, [None]+n_input)#Here the [None]+n_input is needed since the placeholder needs to store multiple different images of size 40x40x3


In [24]:
#Copied diretly from CNN Notebook
# Store layers weights & biases
weights = {
    # 5x5 conv, 3 inputs, 16 outputs
    'encoder_h1': tf.Variable(tf.truncated_normal([3, 3, 3, n_channel1], stddev=0.01)),
    # 5x5 conv, 16 inputs, 32 outputs
    'encoder_h2': tf.Variable(tf.truncated_normal([3, 3, n_channel1, n_channel2], stddev=0.01)),
    # 5x5 conv, 3 inputs, 16 outputs
    'decoder_h1': tf.Variable(tf.truncated_normal([3, 3, n_channel1, n_channel2], stddev=0.01)),
    # 5x5 conv, 16 inputs, 32 outputs
    'decoder_h2': tf.Variable(tf.truncated_normal([3, 3, 3, n_channel1], stddev=0.01))
}

biases = {
    'encoder_h1': tf.Variable(tf.truncated_normal([n_channel1], stddev=0.01)),
    'encoder_h2': tf.Variable(tf.truncated_normal([n_channel2], stddev=0.01)),
    'decoder_h1': tf.Variable(tf.truncated_normal([n_channel1], stddev=0.01)),
    'decoder_h2': tf.Variable(tf.truncated_normal([3], stddev=0.01))
}

In [25]:
def conv_model(x,weights,biases):
    input_layer=tf.reshape(x,[-1,40,40,3])
    conv1=tf.nn.conv2d(x, filter=weights['encoder_h1'], strides=[1, 2, 2, 1], padding='SAME')
    conv1=tf.nn.bias_add(conv1, biases['encoder_h1'])
    conv1=tf.nn.relu(conv1)
    pool1= tf.nn.max_pool(conv1, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
    conv2=tf.nn.conv2d(pool1, filter=weights['encoder_h2'], strides=[1, 2, 2, 1], padding='SAME')
    conv2=tf.nn.bias_add(conv2, biases['encoder_h2'])
    conv2=tf.nn.relu(conv2)
    pool2= tf.nn.max_pool(conv2, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
    #We do not understand the pooling documentation yet. we have a tenuous grasp of it. Also stides are always taken as 2 since that is best practice and we are not able to visualize properly in our heads. Outputshape is also just pretty much copied . Will look into the documentation for a better ideaof conv2d transpose
    out_dim1 = tf.stack([tf.shape(x)[0], 10, 10, n_channel1])
    deconv1=tf.nn.conv2d_transpose(pool2,weights['decoder_h1'],out_dim1,strides=[1,2,2,1],padding='SAME')
    deconv1=tf.nn.bias_add(deconv1,biases['decoder_h1'])
    deconv1=tf.nn.relu(deconv1)
    out_dim2 = tf.stack([tf.shape(x)[0], 20, 20 ,3])
    depool1=tf.image.resize_nearest_neighbor(deconv1, size=tf.stack([20, 20]))
    deconv2=tf.nn.conv2d_transpose(deconv1,weights['decoder_h2'],out_dim2,strides=[1,2,2,1],padding='SAME')
    deconv2=tf.nn.bias_add(deconv2,biases['decoder_h2'])
    deconv2=tf.nn.relu(deconv2)
    depool2=tf.image.resize_nearest_neighbor(deconv2, size=tf.stack([40, 40]))
    return depool2

In [26]:
pred=conv_model(X,weights,biases)
y= X
print(y)
print(pred)

# Define loss and optimizer
cost = tf.reduce_mean(tf.pow(pred - y, 2))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

Tensor("Placeholder:0", shape=(?, 40, 40, 3), dtype=float32)
Tensor("ResizeNearestNeighbor_23:0", shape=(?, 40, 40, 3), dtype=float32)


ValueError: Incompatible shapes between op input and calculated input gradient.  Forward operation: conv2d_transpose_22.  Input index: 2. Original input shape: (?, 3, 3, 64).  Calculated input gradient shape: (?, 5, 5, 64)

In [27]:
#INCOMPLETE. hAVE TO CORRECT SHAPE ERRORS. FINDING IT HARD SINCE WE DONT REALLY UNDERSTAND POOLING